# ICE-NMF Evaluation ml1m

In [7]:
#imports
import numpy as np 
import csv
import re
import sys
import math
import matplotlib.pyplot as plt
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from implicit_data import ImplicitData
import pandas as pd
from BISGD import BISGD
from ICE_NMF import ICE_NMF
from collections import defaultdict
from EvalPrequential import EvalPrequential
from datetime import datetime
import getopt

## Functions


In [8]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [9]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ml1m_gte5.csv'


Data = []
data = pd.read_csv("ml1m_gte5.csv","\t")
print(data)
Data = data.values.tolist()



        UserID  ItemID   Timestamp
0         6040     593   956703954
1         6040    2019   956703977
2         6040     213   956704056
3         6040    3111   956704056
4         6040     912   956704191
...        ...     ...         ...
226305    5950    3363  1046367948
226306    5950     111  1046368241
226307    5950    3196  1046368462
226308    5948    1180  1046437879
226309    4958    1407  1046454443

[226310 rows x 3 columns]


# Full dataset

In [10]:
Data_Transpose = np.transpose(Data[:50000])
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



overlap = {0.6,0.7,0.8}
num_clusters = {8,16,24}
cl_num_iterations = 10
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4


In [11]:
%%capture out

tuning = {}

for num in num_clusters:
    tuning[num] = {}
    
    
    for val in overlap:

        print("")
        print("num_clusters = ", num, ", overlap = ", val)
        model = ICE_NMF(ImplicitData([],[]), num, val, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 100)
        eval = EvalPrequential(model,stream, metrics = ["Recall@20"])

        start_recommend = datetime.now()
        print('start time', start_recommend)

        results=eval.EvaluateTime(0,stream.size, 100)

        end_recommend = datetime.now()
        print('end time', end_recommend)

        tempo = end_recommend - start_recommend

        tuning[num][val] = np.mean(results['Recall@20'])
    
        print('npmean(resuls[Recall@20])', tuning[num][val])
        print('run time', tempo)
        #print('get tuple',np.mean(results['time_get_tuple']))
        print('recommend',np.mean(results['time_recommend']))
        #print('eval_point',np.mean(results['time_eval_point']))
        print('update',np.mean(results['time_update'])) 

In [12]:
print(out)


num_clusters =  8 , overlap =  0.6
start time 2023-11-23 11:50:24.196539
end time 2023-11-23 11:51:27.223509
npmean(resuls[Recall@20]) 0.08823529411764706
run time 0:01:03.026970
recommend 0.018119482900582108
update 0.0010697458457946778

num_clusters =  8 , overlap =  0.7
start time 2023-11-23 11:51:27.259524
end time 2023-11-23 11:52:35.639903
npmean(resuls[Recall@20]) 0.08906882591093117
run time 0:01:08.380379
recommend 0.01802161492799458
update 0.0011835494518280028

num_clusters =  8 , overlap =  0.8
start time 2023-11-23 11:52:35.665501
end time 2023-11-23 11:53:41.735236
npmean(resuls[Recall@20]) 0.08278867102396514
run time 0:01:06.069735
recommend 0.018203794566634433
update 0.0011484846067428588

num_clusters =  16 , overlap =  0.6
start time 2023-11-23 11:53:41.766588
end time 2023-11-23 11:55:25.742226
npmean(resuls[Recall@20]) 0.0743801652892562
run time 0:01:43.975638
recommend 0.03906157243350321
update 0.0016952106618881226

num_clusters =  16 , overlap =  0.7
start

In [13]:
tuning

{8: {0.6: 0.08823529411764706,
  0.7: 0.08906882591093117,
  0.8: 0.08278867102396514},
 16: {0.6: 0.0743801652892562,
  0.7: 0.10810810810810811,
  0.8: 0.08757637474541752},
 24: {0.6: 0.08757637474541752,
  0.7: 0.10756972111553785,
  0.8: 0.09885931558935361}}